### Download Whisper

In [ ]:
#避免Colab断开连接的方法：
#在网页内按键Ctrl+Shift+I，跳到控制台界面，输入以下的内容：
#function ConnectButton(){
    #console.log("Connect pushed"); 
    #document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()}
#setInterval(ConnectButton,60000);
#记得删掉前面的#

In [ ]:
!git clone https://github.com/AlexandaJerry/whisper-vits-japanese

Cloning into 'whisper-vits-japanese'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 135 (delta 7), reused 12 (delta 3), pack-reused 108
Receiving objects: 100% (135/135), 43.80 MiB | 22.73 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
%%capture
!pip install git+https://github.com/openai/whisper.git

In [ ]:
!cp -r /content/whisper-vits-japanese/whisper/transcribe.py /usr/local/lib/python3.7/dist-packages/whisper
!cp -r /content/whisper-vits-japanese/whisper/utils.py /usr/local/lib/python3.7/dist-packages/whisper

In [ ]:
import whisper

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

### Prepare Data for Whisper ASR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###此处是唯一需要自己改动的地方：自己的音频zip路径

In [ ]:
#目前支持日语，换成别语言需替换text文件夹cleaner和symbols以及config文件夹中isla_base.json的cleaner参数
#这里只需用新路径替换掉/content/drive/MyDrive/isla_base/isla.zip即可(音频需为wav)
!unzip -j /content/drive/MyDrive/isla_base/isla.zip "*/*.wav" -d /content/whisper-vits-japanese/audio

In [ ]:
#如果没有训练数据只想体验的话，这里只能暂时借用下White老师的数据集了，作为入门材料真是万分感谢
%cd /content/whisper-vits-japanese
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1H6aqdGP-h-MT7XAVk870Ql4d3QrJT-7o' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1H6aqdGP-h-MT7XAVk870Ql4d3QrJT-7o" -O "isla.zip" && rm -rf /tmp/cookies.txt
!unzip -j /content/whisper-vits-japanese/isla.zip "*/*.wav" -d /content/whisper-vits-japanese/audio

In [ ]:
#识别过程总共花费50分钟(大概是音频总长的三分之一到一半左右)
!whisper -i /content/whisper-vits-japanese/audio -o /content/whisper-vits-japanese/srt_files --language Japanese

In [ ]:
#auto.py会根据Whisper导出的srt自动切片音频、转写文本、转采样率为22050HZ
#最后会把适配VIST的数据格式汇总在/content/whisper-vits-japanese/filelists
#然后音频数据会被放在/content/whisper-vits-japanese/sliced_audio文件夹里
!pip install pydub
%cd /content/whisper-vits-japanese
!python auto.py

### Install Requirements of VITS

In [ ]:
!pip install -r requirements.txt
!sudo apt-get install espeak -y

### Alignment and Text Conversion

In [ ]:
import os
path = "/content/whisper-vits-japanese"
os.chdir(path)
print(os.getcwd())

/content/whisper-vits-japanese


In [ ]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

In [ ]:
import os
path = "/content/whisper-vits-japanese"
os.chdir(path)
print(os.getcwd())

/content/whisper-vits-japanese


In [ ]:
#如果输出提示nophoneme这种情况，检查下/content/whisper-vits-japanese/filelists的两个txt文件里有没有出现英文转写
!python preprocess.py --text_index 1 --text_cleaners japanese_cleaners --filelists /content/whisper-vits-japanese/filelists/train_filelist.txt /content/whisper-vits-japanese/filelists/val_filelist.txt

START: /content/whisper-vits-japanese/filelists/train_filelist.txt
Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"
dic.tar.gz: 100% 22.6M/22.6M [00:02<00:00, 10.9MB/s]
Extracting tar file /usr/local/lib/python3.7/dist-packages/pyopenjtalk/dic.tar.gz
START: /content/whisper-vits-japanese/filelists/val_filelist.txt


### Train

In [ ]:
!python train.py -c configs/isla_base.json -m isla_base

### Inference

In [ ]:
#如果出现大批量的log日志显示在输出的话
#在"代码执行程序"下拉菜单选择"重新启动代码程序"
#再从该代码框开始，重新运行进行推断和输出语音
import os
path = "/content/whisper-vits-japanese"
os.chdir(path)
print(os.getcwd())

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

/content/whisper-vits-japanese


In [ ]:
hps = utils.get_hparams_from_file("/content/whisper-vits-japanese/configs/isla_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

#下面的这个isla_base/G_6000.pth需要换成目标文件夹里数字最大的G_????.pth
_ = utils.load_checkpoint("/content/whisper-vits-japanese/logs/isla_base/G_????.pth", net_g, None)

In [ ]:
stn_tst = get_text("ビエザイゼリ…ーファーディエン", hps) #别在这里发癫 停顿可用…、
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

### Save Materials and Checkpoints to Drive for Future Usage

In [45]:
#把checkpoint存入google drive(这里用了RT防止递归式复制到目标文件夹)
!cp -RT /content/whisper-vits-japanese/logs/ /content/drive/MyDrive/logs/
#把音频文件和对应抄本存入google drive
!cp -RT /content/whisper-vits-japanese/sliced_audio/ /content/drive/MyDrive/sliced_audio/
!cp -RT /content/whisper-vits-japanese/filelists/ /content/drive/MyDrive/filelists/

### Restart from Lastest Checkpoint in the Next Time



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [44]:
#把google drive的checkpoint恢复到文件夹
!cp -r /content/drive/MyDrive/logs/isla_base/** /content/whisper-vits-japanese/logs/isla_base/
#把音频文件和对应抄本恢复到文件夹
!cp -r /content/drive/MyDrive/sliced_audio/** /content/whisper-vits-japanese/sliced_audio/
!cp -r /content/drive/MyDrive/filelists/** /content/whisper-vits-japanese/filelists/

In [ ]:
%cd /content/whisper-vits-japanese
!pip install -r requirements.txt
!sudo apt-get install espeak -y
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..
!python train.py -c configs/isla_base.json -m isla_base